In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

In [2]:
books = pd.read_csv("data/clean/books_clean.csv")
ratings = pd.read_csv("data/clean/ratings_clean.csv")
users = pd.read_csv("data/clean/users_clean.csv")

In [3]:
users.country.isnull().sum()

4543

In [4]:
#unifying the books with ratings tables on isbn field
#number of entries in books table
len(books)

266732

In [16]:
#ratings table size
len(ratings)

433671

In [17]:
books_with_ratings = pd.merge(ratings, books, on='isbn')

In [18]:
#Checking if there are any rows with title/author missing
books_with_ratings.book_title.isnull().sum()

0

In [19]:
books_with_ratings.book_author.isnull().sum()

0

In [20]:
books_with_ratings.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher
0,276726,0155061224,5,Rites of Passage,Judith Rae,2001.0,Heinle
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999.0,Cambridge University Press
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001.0,Cambridge University Press
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001.0,Doubleday
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,2001.0,Doubleday


In [21]:
#unifying ISBN numbers
#looking the number of books with multiple isbn numbers
multiple_isbns = books_with_ratings.groupby('book_title').isbn.nunique()
multiple_isbns.value_counts()

1     123409
2       7733
3       1426
4        457
5        179
6         81
7         37
8         24
9          8
10         6
16         2
15         2
14         2
13         2
12         2
11         2
Name: isbn, dtype: int64

In [22]:
has_multiple_isbns = multiple_isbns.where(multiple_isbns > 1) #sets NaN for those with just 1 isb

In [23]:
#removing the ones with just one isbn
has_multiple_isbns.dropna(inplace=True)

In [25]:
len(has_multiple_isbns)

9963

In [26]:
#Create dictionary for books with multiple isbns
def make_isbn_dict(df):
    title_isbn_dict = {}
    for title in has_multiple_isbns.index:
        isbn_series = df.loc[df.book_title == title].isbn.unique() # returns only the unique ISBNs
        title_isbn_dict[title] = isbn_series.tolist()
    return title_isbn_dict

%time dict_unique_isbn = make_isbn_dict(books_with_ratings)

CPU times: user 5min 41s, sys: 1.79 s, total: 5min 43s
Wall time: 5min 45s


In [27]:
with open('multiple_isbn_dict.pickle', 'wb') as handle:
    pickle.dump(dict_unique_isbn, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
# LOAD isbn_dict back into namespace
with open('multiple_isbn_dict.pickle', 'rb') as handle:
    multiple_isbn_dict = pickle.load(handle)

In [29]:
#checking the number of entries in the dictiionaries
len(multiple_isbn_dict)

9963

In [30]:
# Adding 'unique_isbn' column to 'books_with_ratings' dataframe that includes the first ISBN if multiple ISBNS,or the single unique isbn
def add_unique_isbn_col(df):
    df['unique_isbn'] = df.apply(lambda row: multiple_isbn_dict[row.book_title][0] if row.book_title in multiple_isbn_dict.keys() else row.isbn, axis=1)
    return df

%time books_with_ratings = add_unique_isbn_col(books_with_ratings)

CPU times: user 9.28 s, sys: 101 ms, total: 9.39 s
Wall time: 9.51 s


In [31]:
#checking an example to see that the unique column has been added 
books_with_ratings[books_with_ratings.book_title == "Jane Eyre"].head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn
40062,387,1590071212,9,Jane Eyre,Charlotte Bronte,2002.0,New Millenium Audio,1590071212
65551,2718,0451518845,10,Jane Eyre,Charlotte Bronte,1999.0,New Amer Library Classics,1590071212
65552,70205,0451518845,9,Jane Eyre,Charlotte Bronte,1999.0,New Amer Library Classics,1590071212
65553,218121,0451518845,10,Jane Eyre,Charlotte Bronte,1999.0,New Amer Library Classics,1590071212
109854,8370,0451523326,8,Jane Eyre,Charlotte Bronte,1988.0,Signet Classics,1590071212


In [32]:
#joining the users table on hte user_id field
books_users_ratings = books_with_ratings.merge(users, on='user_id')

In [33]:
books_users_ratings.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn,age,city,state,country
0,276726,0155061224,5,Rites of Passage,Judith Rae,2001.0,Heinle,0155061224,34.786876,seattle,washington,usa
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999.0,Cambridge University Press,052165615X,16.000000,rijeka,n/a,croatia
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001.0,Cambridge University Press,0521795028,16.000000,rijeka,n/a,croatia
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001.0,Doubleday,038550120X,34.786876,torrance,california,usa
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,2001.0,Doubleday,038550120X,34.786876,NaN,n/a,n/a


In [34]:
books_users_ratings.to_csv("data/clean/books_users_ratings.csv", index=False)